In [1]:
#!pip install unstructured[pdf]

In [2]:
#from unstructured.partition.pdf import partition_pdf

#elements = partition_pdf(filename="llama2.pdf")

In [3]:
#!pip install pdfplumber

In [4]:
import pdfplumber

def extract_text_and_tables(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        extracted_text = ""
        tables = []
        t=[]
        for page in pdf.pages:
            # Extract text
            #extracted_text += page.extract_text()
            t.append(page.extract_text())

            # Extract tables
            page_tables = page.extract_tables()
            tables.extend(page_tables)
            
    return extracted_text, tables, t

# Example usage
pdf_path="llama2.pdf"
text, tables, pages = extract_text_and_tables(pdf_path)

# Print extracted text
#print("Extracted Text:\n", text)

# Print extracted tables
#for i, table in enumerate(tables):
#    print(f"Table {i+1}:")
#    for row in table:
#        print(row)

In [2]:
print(pages[50])

Model Size GSM8k MATH
7B 6.8 3.0
MPT
30B 15.2 3.1
7B 6.8 2.3
Falcon
40B 19.6 5.5
7B 11.0 2.9
13B 17.8 3.9
Llama1
33B 35.6 7.1
65B 50.9 10.6
7B 14.6 2.5
13B 28.7 3.9
Llama2
34B 42.2 6.24
70B 56.8 13.5
Table25: Comparisontootheropen-sourcemodelsonmathematicalreasoningtasks,GSM8kandMATH
(maj1@1isreported).
MathematicalReasoning. InTable25,wereportresultsforLlama2andotheropen-sourcedatasetsonthe
GSM8kandMATHtasks.
A.3 AdditionalDetailsforFine-tuning
A.3.1 DetailedStatisticsofMetaHumanPreferenceData
Table26showsdetailedstatisticsonMetahumanpreferencedata. Intotal,wecollected14batchesofhuman
preferencedata(i.e.,MetaSafety+Helpfulness)onaweeklybasis,consistingofover1millionbinarymodel
generationcomparisons. Ingeneral,laterbatchescontainmoresamplesasweonboardmoreannotatorsover
timeandtheannotatorsalsobecomemorefamiliarwiththetasksandthushavebetterworkefficiency. We
alsointentionallycollectmoremulti-turnsamplestoincreasethecomplexityofRLHFdataandthusthe
averagenumberoftokenspersamplealsoincreas

In [2]:
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "llama2.pdf"

--2024-08-07 10:35:39--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2307.09288 [following]
--2024-08-07 10:35:39--  http://arxiv.org/pdf/2307.09288
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘llama2.pdf’

llama2.pdf          100%[===================>]  13.03M  --.-KB/s    in 0.09s   

2024-08-07 10:35:39 (146 MB/s) - ‘llama2.pdf’ saved [13661300/13661300]



In [ ]:
https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Table%20Transformer/Inference_with_Table_Transformer_(TATR)_for_parsing_tables.ipynb

In [5]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Patch
import io
from PIL import Image, ImageDraw
import numpy as np
import csv
import pandas as pd

from torchvision import transforms

from transformers import AutoModelForObjectDetection
import torch
import openai
import os
import fitz

device = "cuda" if torch.cuda.is_available() else "cpu"

OPENAI_API_TOKEN = "sk-<your-openai-api-token>"
openai.api_key = OPENAI_API_TOKEN

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
pdf_file = "llama2.pdf"

# Split the base name and extension
output_directory_path, _ = os.path.splitext(pdf_file)

if not os.path.exists(output_directory_path):
    os.makedirs(output_directory_path)

# Open the PDF file
pdf_document = fitz.open(pdf_file)

# Iterate through each page and convert to an image
for page_number in range(pdf_document.page_count):
    # Get the page
    page = pdf_document[page_number]

    # Convert the page to an image
    pix = page.get_pixmap()

    # Create a Pillow Image object from the pixmap
    image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Save the image
    image.save(f"./{output_directory_path}/page_{page_number + 1}.png")

# Close the PDF file
pdf_document.close()

In [ ]:
https://docs.llamaindex.ai/en/v0.9.48/examples/multi_modal/multi_modal_pdf_tables.html

In [9]:
import base64
import requests

# OpenAI API key


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "./llama2/page_2.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "extract the contents of the image as markdown"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 3000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9u2A3GrLzwjylNl2sLY4oPpERxHII', 'object': 'chat.completion', 'created': 1723141703, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```markdown\n# Contents\n\n1. Introduction ......................................................... 3\n\n2. Pretraining .......................................................... 5  \n   2.1 Pretraining Data ................................................. 5  \n   2.2 Training Details ................................................. 5  \n   2.3 LLAMA 2 Pretrained Model Evaluation ............................... 7  \n\n3. Fine-tuning ........................................................... 8  \n   3.1 Supervised Fine-Tuning (SFT) ..................................... 9  \n   3.2 Reinforcement Learning with Human Feedback (RLHF) .................. 9  \n   3.3 System Message for Multi-Turn Consistency ........................ 16  \n   3.4 RLHF Results .............................

In [5]:
!pip install llama-index qdrant_client pyMuPDF tools frontend git+https://github.com/openai/CLIP.git easyocr

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-zl_ccvyf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-zl_ccvyf
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 53.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.

In [ ]:
class MaxResize(object):
    def __init__(self, max_size=800):
        self.max_size = max_size

    def __call__(self, image):
        width, height = image.size
        current_max_size = max(width, height)
        scale = self.max_size / current_max_size
        resized_image = image.resize(
            (int(round(scale * width)), int(round(scale * height)))
        )

        return resized_image


detection_transform = transforms.Compose(
    [
        MaxResize(800),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

structure_transform = transforms.Compose(
    [
        MaxResize(1000),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

# load table detection model
# processor = TableTransformerImageProcessor(max_size=800)
model = AutoModelForObjectDetection.from_pretrained(
    "microsoft/table-transformer-detection", revision="no_timm"
).to(device)

# load table structure recognition model
# structure_processor = TableTransformerImageProcessor(max_size=1000)
structure_model = AutoModelForObjectDetection.from_pretrained(
    "microsoft/table-transformer-structure-recognition-v1.1-all"
).to(device)


# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    width, height = size
    boxes = box_cxcywh_to_xyxy(out_bbox)
    boxes = boxes * torch.tensor(
        [width, height, width, height], dtype=torch.float32
    )
    return boxes


def outputs_to_objects(outputs, img_size, id2label):
    m = outputs.logits.softmax(-1).max(-1)
    pred_labels = list(m.indices.detach().cpu().numpy())[0]
    pred_scores = list(m.values.detach().cpu().numpy())[0]
    pred_bboxes = outputs["pred_boxes"].detach().cpu()[0]
    pred_bboxes = [
        elem.tolist() for elem in rescale_bboxes(pred_bboxes, img_size)
    ]

    objects = []
    for label, score, bbox in zip(pred_labels, pred_scores, pred_bboxes):
        class_label = id2label[int(label)]
        if not class_label == "no object":
            objects.append(
                {
                    "label": class_label,
                    "score": float(score),
                    "bbox": [float(elem) for elem in bbox],
                }
            )

    return objects

def detect_and_crop_save_table(
    file_path, cropped_table_directory="./table_images/"
):
    image = Image.open(file_path)

    filename, _ = os.path.splitext(file_path.split("/")[-1])

    if not os.path.exists(cropped_table_directory):
        os.makedirs(cropped_table_directory)

    # prepare image for the model
    # pixel_values = processor(image, return_tensors="pt").pixel_values
    pixel_values = detection_transform(image).unsqueeze(0).to(device)

    # forward pass
    with torch.no_grad():
        outputs = model(pixel_values)

    # postprocess to get detected tables
    id2label = model.config.id2label
    id2label[len(model.config.id2label)] = "no object"
    detected_tables = outputs_to_objects(outputs, image.size, id2label)

    print(f"number of tables detected {len(detected_tables)}")

    for idx in range(len(detected_tables)):
        #   # crop detected table out of image
        cropped_table = image.crop(detected_tables[idx]["bbox"])
        cropped_table.save(f"./{cropped_table_directory}/{filename}_{idx}.png")

def plot_images(image_paths):
    images_shown = 0
    plt.figure(figsize=(16, 9))
    for img_path in image_paths:
        if os.path.isfile(img_path):
            image = Image.open(img_path)

            plt.subplot(2, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 9:
                break
                

In [ ]:
for file_path in retrieved_images:
    detect_and_crop_save_table(file_path)

In [ ]:
# Read the cropped tables
image_documents = SimpleDirectoryReader("./table_images/").load_data()

In [ ]:
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index import SimpleDirectoryReader

# put your local directore here
#documents_images_v2 = SimpleDirectoryReader("./llama2/").load_data()

In [ ]:
openai_mm_llm = OpenAIMultiModal(
    model="gpt-4-vision-preview", api_key=OPENAI_API_TOKEN, max_new_tokens=1500
)


In [ ]:
response = openai_mm_llm.complete(
    prompt="Compare llama2 with llama1?",
    image_documents=image_documents,
)

print(response)

In [ ]:
import glob

table_images_paths = glob.glob("./table_images/*.png")

In [ ]:
plot_images(table_images_paths)